In [1]:
"""
if 'google.colab' in sys.modules:  # colab環境特有の処理_初回のみ
  # Google Driveのマウント
  from google.colab import drive
  drive.mount('/content/drive')

  !pip install --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules' \
   -r '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/requirements.txt' \
   --ignore-installed

  !pip install --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules' \
   transformers -U
  !pip install gensim==4.0.1 --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules'
  !pip install pytorch_memlab --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules'
"""

"\nif 'google.colab' in sys.modules:  # colab環境特有の処理_初回のみ\n  # Google Driveのマウント\n  from google.colab import drive\n  drive.mount('/content/drive')\n\n  !pip install --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules'    -r '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/requirements.txt'    --ignore-installed\n\n  !pip install --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules'    transformers -U\n  !pip install gensim==4.0.1 --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules'\n  !pip install pytorch_memlab --target '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules'\n"

In [2]:
!pip install pytorch_memlab

In [3]:
import sys
if 'google.colab' in sys.modules:  # colab特有の処理_2回目以降
  # Google Driveのマウント
  from google.colab import drive
  drive.mount('/content/drive')

  # データセットをDriveから取得
  !mkdir -p 'input'
  !cp -r '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/00_input' '/content/input'

  # ライブラリのパス指定
  sys.path.append('/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from pathlib import Path

# input
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    DATA_DIR = Path('../input/commonlitreadabilityprize/')

elif 'google.colab' in sys.modules: # Colab環境
    !mkdir 'input' -p
    !cp '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/00_input/commonlitreadabilityprize/' './input' -r
    DATA_DIR = Path('/content/input/commonlitreadabilityprize')

else:
    DATA_DIR = Path('../00_input/commonlitreadabilityprize/')

In [5]:
from pathlib import Path

# pre-trained model
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    PRE_TRAINED_MODEL_DIR = '../input/roberta-transformers-pytorch/roberta-large'
elif 'google.colab' in sys.modules: # Colab環境
    PRE_TRAINED_MODEL_DIR = 'roberta-base' # 仮で、毎回DLする想定のモデル名を指定。あとで変更予定。
else:
    PRE_TRAINED_MODEL_DIR = 'roberta-base'

In [6]:
from pathlib import Path

# pre-trained model
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    PRE_TRAINED_MODEL_DIR = '../input/roberta-transformers-pytorch/roberta-large'
elif 'google.colab' in sys.modules: # Colab環境
    PRE_TRAINED_MODEL_DIR = 'roberta-base' # 仮で、毎回DLする想定のモデル名を指定。あとで変更予定。
else:
    PRE_TRAINED_MODEL_DIR = 'roberta-base'

In [7]:
UPLOAD_DIR = Path('/content/model')
EX_NO = '039-train-03'  # 実験番号などを入れる、folderのpathにする
USERID = 'calpis10000'

In [8]:
import subprocess
import shlex

def gpuinfo():
    """
    Returns size of total GPU RAM and used GPU RAM.

    Parameters
    ----------
    None

    Returns
    -------
    info : dict
        Total GPU RAM in integer for key 'total_MiB'.
        Used GPU RAM in integer for key 'used_MiB'.
    """

    command = 'nvidia-smi -q -d MEMORY | sed -n "/FB Memory Usage/,/Free/p" | sed -e "1d" -e "4d" -e "s/ MiB//g" | cut -d ":" -f 2 | cut -c2-'
    commands = [shlex.split(part) for part in command.split(' | ')]
    for i, cmd in enumerate(commands):
        if i==0:
            res = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        else:
            res = subprocess.Popen(cmd, stdin=res.stdout, stdout=subprocess.PIPE)
    total, used = map(int, res.communicate()[0].decode('utf-8').strip().split('\n'))
    info = {'total_MiB':total, 'used_MiB':used}
    return info


# Overview
This nb is based on copy from https://www.kaggle.com/andretugan/lightweight-roberta-solution-in-pytorch .

Acknowledgments(from base nb): 
some ideas were taken from kernels by [Torch](https://www.kaggle.com/rhtsingh) and [Maunish](https://www.kaggle.com/maunish).

In [9]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW # optimizer
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup # scheduler
from pytorch_memlab import profile
import pytorch_memlab
from pytorch_memlab import MemReporter

from sklearn.model_selection import KFold

import gc
gc.enable()

In [10]:
NUM_FOLDS = 5 # K Fold
NUM_EPOCHS = 5 # Epochs
BATCH_SIZE = 16 # Batch Size
MAX_LEN = 248 # ベクトル長
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)] # schedulerの何らかの設定？
ROBERTA_PATH = PRE_TRAINED_MODEL_DIR # roberta pre-trainedモデル(モデルとして指定)
TOKENIZER_PATH = PRE_TRAINED_MODEL_DIR # roberta pre-trainedモデル(Tokenizerとして指定)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # cudaがなければcpuを使えばいいじゃない

In [11]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True# cudnnによる最適化で結果が変わらないためのおまじない 

In [12]:
# train, testを読む
train_df = pd.read_csv(DATA_DIR/"train.csv")

# Remove incomplete entries if any.
train_df.drop(train_df[(train_df.target == 0) & (train_df.standard_error == 0)].index,
              inplace=True)
train_df.reset_index(drop=True, inplace=True)

test_df = pd.read_csv(DATA_DIR/"test.csv")
submission_df = pd.read_csv(DATA_DIR/"sample_submission.csv")

In [13]:
# tokenizerを指定
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

# Dataset

In [14]:
# Dataset用のClass。おそらく、trainとtestでインスタンスを生成し、DataFrameと同じように扱えるような思想。
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only # Testデータ用フラグ
        self.text = df.excerpt.tolist() # 分析対象カラムをlistにする。(分かち書きではなく、Seriesをlistへ変換するような処理)
        #self.text = [text.replace("\n", " ") for text in self.text] # 単語単位で分かち書きする場合
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32) # trainのみ、targetをtensorに変換
    
        self.encoded = tokenizer.batch_encode_plus( # textをtokenize
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True, # 最大長を超える文字は切り捨て
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index): # 変換結果を返す
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

# Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [15]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH) # pretrainedからconfigを読み込み
        config.update({"output_hidden_states":True, # config更新: embedding層を抽出
                       "hidden_dropout_prob": 0.0, # config更新: dropoutしない
                       "layer_norm_eps": 1e-7}) # config更新: layer normalizationのepsilon                      
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config) # cpuで処理する
            
        self.attention = nn.Sequential(# attentionレイヤー            
            nn.Linear(768, 512), # 1024(元は768)は、ベースとなる学習済みモデルの重みをハードコードしてる。イケてないので、configから取得する感じにしたい。     
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )

        self.regressor = nn.Sequential( # 出力レイヤー                    
            nn.Linear(768, 1)                        
        )
        
    %profile
    def forward(self, input_ids, attention_mask):
        #print('model: get_roberta_out: Start', gpuinfo())                 
        roberta_output = self.roberta(input_ids=input_ids, # robertaに入力データを流し、出力としてrobertaモデル(layerの複合体)を得る
                                      attention_mask=attention_mask)     
        #print('model: get_roberta_out: Done', gpuinfo())                 

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1] # robertaモデルの最後のlayerを得る

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base). # 
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states) # robertaの最後のlayerをattentionへ入力し、出力として重みを得る
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) # 重み×最後の層を足し合わせて文書ベクトルとする。
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector) # 文書ベクトルを線形層に入力し、targetを出力する

default


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/basic.py:300: UserWarning: %profile is now deprecated. Please use get_ipython().profile instead.
  warn("%profile is now deprecated. Please use get_ipython().profile instead.")


In [16]:
# 評価指標(MSE)の計算。最終的に、ルートしてRMSEにすると思われる。
def eval_mse(model, data_loader):
    """Evaluates the mean squared error of the |model| on |data_loader|"""
    model.eval() # evalモードを選択。Batch Normとかdropoutをしなくなる           
    mse_sum = 0

    with torch.no_grad(): # 勾配の計算をしないBlock
        for batch_num, (input_ids, attention_mask, target) in enumerate(data_loader): # data_loaderからinput, attentin_mask, targetをbatchごとに取り出す
            input_ids = input_ids.to(DEVICE)   
            attention_mask = attention_mask.to(DEVICE)   
            target = target.to(DEVICE)           
            
            pred = model(input_ids, attention_mask) # 取得した値をモデルへ入力し、出力として予測値を得る。

            mse_sum += nn.MSELoss(reduction="sum")(pred.flatten(), target).item() # 誤差の合計を得る(Batchごとに計算した誤差を足し上げる)
    del input_ids
    del attention_mask
    del target
  
    return mse_sum / len(data_loader.dataset) # 誤差の合計をdataset長で除し、mseを取得＆返す

In [17]:
# 推論結果を返す
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval() # evalモード(dropout, batch_normしない)

    result = np.zeros(len(data_loader.dataset)) # 結果をdataset長のzero配列として用意
    index = 0
    
    with torch.no_grad(): # 勾配の計算をしないblock(inputすると、現状の重みによる推論結果を返す)
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader): # data_loaderからbatchごとにinputを得る
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask) # modelにinputを入力し、予測結果を得る。

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu") # result[index ~ predの長さ]へ、予測結果を格納
            index += pred.shape[0] # indexを更新

    return result # 全batchで推論が終わったら、結果を返す

In [18]:
# 学習
def train(model, # モデル
          model_path, # モデルのアウトプット先
          train_loader, # train-setのdata_loader
          val_loader, # valid-setのdata_loader
          optimizer, # optimizer
          scheduler=None, # scheduler, デフォルトはNone
          num_epochs=NUM_EPOCHS # epoch数、notebook冒頭で指定した値
         ):    
    
    best_val_rmse = None
    best_epoch = 0
    step = 0
    last_eval_step = 0
    eval_period = EVAL_SCHEDULE[0][1] # eval期間(って何？) 冒頭で決めたEVAL_SCHEDULEの最初のtupleの[1]を取得

    start = time.time() # 時間計測用

    for epoch in range(num_epochs): # 指定したEpoch数だけ繰り返し
        val_rmse = None         

        for batch_num, (input_ids, attention_mask, target) in enumerate(train_loader): # train_loaderからinput, targetを取得
            input_ids = input_ids.to(DEVICE) # inputをDEVICEへ突っ込む
            attention_mask = attention_mask.to(DEVICE)       
            target = target.to(DEVICE)   
            #print(f'train_{epoch}_{batch_num}: get_input: Done', gpuinfo())                 

            optimizer.zero_grad() # 勾配を初期化
            
            model.train() # 学習モード開始

            #print(f'train_{epoch}_{batch_num}: get_pred: Start', gpuinfo())                 
            pred = model(input_ids, attention_mask) # input,attention_maskを入力し、予測結果を得る
            #print(f'train_{epoch}_{batch_num}: get_pred: Done', gpuinfo())  

            #print(f'train_{epoch}_{batch_num}: get_loss: Start', gpuinfo())                 
            mse = nn.MSELoss(reduction="mean")(pred.flatten(), target) # 予測結果からMSEを得る(これが損失関数となる?)
            #print(f'train_{epoch}_{batch_num}: get_loss: Done', gpuinfo())                 

            mse.backward() # 誤差逆伝播法により勾配を得る
            optimizer.step() # 重みを更新する

            if scheduler:
                scheduler.step() # schedulerが与えられた場合は、schedulerの学習率更新
            
            if step >= last_eval_step + eval_period: # batchを回すごとにstepを増やしていって、「前回evalしたstep + eval_period(16)」を超えたら実行。
                # Evaluate the model on val_loader.
                elapsed_seconds = time.time() - start # 経過時間
                num_steps = step - last_eval_step # 経過ステップ数
                print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                last_eval_step = step # 前回stepの更新
                
                val_rmse = math.sqrt(eval_mse(model, val_loader)) # valid-setによるrmse計算                           

                print(f"Epoch: {epoch} batch_num: {batch_num}", 
                      f"val_rmse: {val_rmse:0.4}") # epoch, batch, score

                for rmse, period in EVAL_SCHEDULE: # eval_periodをvalid-rmseで切り替える処理
                    if val_rmse >= rmse: # valid rmseをEVAL_SCHEDULEと比較し、0項 > valid rmseとなるまで回す : EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
                        eval_period = period # eval_periodを更新
                        break                               

                if not best_val_rmse or val_rmse < best_val_rmse: # 初回(best_val_rmse==None), またはbest_val_rmseを更新したらモデルを保存する
                    best_val_rmse = val_rmse
                    best_epoch = epoch
                    torch.save(model.state_dict(), model_path) # 最高の自分を保存
                    print(f"New best_val_rmse: {best_val_rmse:0.4}")
                else:       
                    print(f"Still best_val_rmse: {best_val_rmse:0.4}", # 更新されない場合は、元のスコアを表示
                          f"(from epoch {best_epoch})")      
                                                  
                start = time.time()
            
            # batchごとにメモリ解放
            #print(f'train_{epoch}_{batch_num}: free_memory: Start', gpuinfo())                  
            del input_ids
            del attention_mask
            del target
            del pred
            del mse
            torch.cuda.empty_cache()
            #print(f'train_{epoch}_{batch_num}: free_memory: Done', gpuinfo())  
                                            
            step += 1
    
    return best_val_rmse

In [19]:
# optimizerの作成
def create_optimizer(model):
    named_parameters = list(model.named_parameters()) # モデルパラメータの取得
    
    roberta_parameters = list(model.roberta.named_parameters())[:-2] # パラメータをroberta用、attention用、regressor用に格納。(直接引っ張ってくる形式に変更)
    attention_parameters = list(model.attention.named_parameters())
    regressor_parameters = list(model.regressor.named_parameters())
        
    attention_group = [params for (name, params) in attention_parameters] # attention用パラメータをリストとして取得
    regressor_group = [params for (name, params) in regressor_parameters] # reg用パラメータをリストとして取得

    parameters = []
    parameters.append({"params": attention_group}) # パラメータをリストに辞書として格納していく
    parameters.append({"params": regressor_group})

    for layer_num, (name, params) in enumerate(roberta_parameters): # レイヤーごとにname, paramsを取得していろんな処理
        weight_decay = 0.0 if "bias" in name else 0.01
        lr = 2e-5

        if layer_num >= 69:        
            lr = 5e-5

        if layer_num >= 133:
            lr = 1e-4
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return AdamW(parameters) # 最終的に、AdamWにパラメータを入力する。

In [20]:
# 実行処理。 KFold & 学習
gc.collect()

SEED = 1000
list_val_rmse = []

kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kfold.split(train_df)):    
    print(f"\nFold {fold + 1}/{NUM_FOLDS}")
    model_path = f"model_{fold + 1}.pth" # model_fold数_.pth
        
    set_random_seed(SEED + fold) # SEEDはfold別に変わるようにする
    
    train_dataset = LitDataset(train_df.loc[train_indices]) # train, validのDataset
    val_dataset = LitDataset(train_df.loc[val_indices])
        
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              drop_last=True, shuffle=True, num_workers=2) # train, validのDataLoader
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=2)    
        
    set_random_seed(SEED + fold) # なんで二回SEEDをセットするのだろう？

    #print('getmodel: Start', gpuinfo())                 
    model = LitModel().to(DEVICE) # modelをDEVICEへぶち込む
    #print('getmodel: Done', gpuinfo())                 

    #print('get_optim_sched: Start', gpuinfo())                 
    optimizer = create_optimizer(model) # optimizerをモデルから作成
    scheduler = get_cosine_schedule_with_warmup( # schedulerを作成
        optimizer,
        num_training_steps=NUM_EPOCHS * len(train_loader),
        num_warmup_steps=50)    
    #print('get_optim_sched: Done', gpuinfo())                 

    list_val_rmse.append(train(model, model_path, train_loader,
                               val_loader, optimizer, scheduler=scheduler)) # 学習開始し、val_rmseのリストを格納

    del model # モデルは保存したので、消す
    gc.collect() 
    
    print("\nPerformance estimates:")
    print(list_val_rmse)
    print("Mean:", np.array(list_val_rmse).mean())
    


Fold 1/5


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 9.67 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9384
New best_val_rmse: 0.9384

16 steps took 9.11 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8438
New best_val_rmse: 0.8438

16 steps took 9.11 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6332
New best_val_rmse: 0.6332

16 steps took 9.12 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6448
Still best_val_rmse: 0.6332 (from epoch 0)

16 steps took 9.14 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5711
New best_val_rmse: 0.5711

16 steps took 9.11 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5284
New best_val_rmse: 0.5284

16 steps took 9.11 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5532
Still best_val_rmse: 0.5284 (from epoch 0)

16 steps took 9.12 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5401
Still best_val_rmse: 0.5284 (from epoch 0)

16 steps took 9.28 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5244
New best_val_rmse: 0.5244

16 steps took 9.11 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.506
New best_val_rmse: 0.506

16 steps 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 9.37 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9605
New best_val_rmse: 0.9605

16 steps took 8.84 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7348
New best_val_rmse: 0.7348

16 steps took 8.84 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7169
New best_val_rmse: 0.7169

16 steps took 8.83 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6634
New best_val_rmse: 0.6634

16 steps took 8.83 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6271
New best_val_rmse: 0.6271

16 steps took 8.84 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6594
Still best_val_rmse: 0.6271 (from epoch 0)

16 steps took 8.82 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5825
New best_val_rmse: 0.5825

16 steps took 8.83 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5705
New best_val_rmse: 0.5705

16 steps took 8.98 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5409
New best_val_rmse: 0.5409

16 steps took 8.83 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5331
New best_val_rmse: 0.5331

16 steps took 8.83 seconds
Epoch: 1 batch

...


16 steps took 9.14 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.021
New best_val_rmse: 1.021

16 steps took 8.63 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.065
Still best_val_rmse: 1.021 (from epoch 0)

16 steps took 8.6 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6681
New best_val_rmse: 0.6681

16 steps took 8.61 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6233
New best_val_rmse: 0.6233

16 steps took 8.61 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6361
Still best_val_rmse: 0.6233 (from epoch 0)

16 steps took 8.6 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6009
New best_val_rmse: 0.6009

16 steps took 8.6 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5342
New best_val_rmse: 0.5342

16 steps took 8.62 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.577
Still best_val_rmse: 0.5342 (from epoch 0)

16 steps took 8.78 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5206
New best_val_rmse: 0.5206

16 steps took 8.61 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5539
Still best_val_rmse: 0.5206 (from epoch 1)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 8.94 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9691
New best_val_rmse: 0.9691

16 steps took 8.42 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8864
New best_val_rmse: 0.8864

16 steps took 8.43 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.8383
New best_val_rmse: 0.8383

16 steps took 8.43 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7232
New best_val_rmse: 0.7232

16 steps took 8.42 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6244
New best_val_rmse: 0.6244

16 steps took 8.41 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6235
New best_val_rmse: 0.6235

16 steps took 8.43 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.561
New best_val_rmse: 0.561

16 steps took 8.41 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6262
Still best_val_rmse: 0.561 (from epoch 0)

16 steps took 8.58 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5432
New best_val_rmse: 0.5432

16 steps took 8.43 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5481
Still best_val_rmse: 0.5432 (from epoch 1)

16 steps took 8.42 seconds


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 8.8 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9208
New best_val_rmse: 0.9208

16 steps took 8.25 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7644
New best_val_rmse: 0.7644

16 steps took 8.26 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6432
New best_val_rmse: 0.6432

16 steps took 8.25 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6147
New best_val_rmse: 0.6147

16 steps took 8.26 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5716
New best_val_rmse: 0.5716

16 steps took 8.26 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5877
Still best_val_rmse: 0.5716 (from epoch 0)

16 steps took 8.27 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6025
Still best_val_rmse: 0.5716 (from epoch 0)

16 steps took 8.25 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5474
New best_val_rmse: 0.5474

16 steps took 8.43 seconds
Epoch: 1 batch_num: 3 val_rmse: 0.5907
Still best_val_rmse: 0.5474 (from epoch 0)

16 steps took 8.25 seconds
Epoch: 1 batch_num: 19 val_rmse: 0.5253
New best_val_rmse: 0.5253

16 steps

In [21]:
print(list_val_rmse)

[0.48490284124832117, 0.4607347237312149, 0.47917388355971047, 0.48806696807311295, 0.4678815096339481]


In [22]:
#rep = MemReporter(model)
#rep.report()

In [23]:
#rep = MemReporter(model.roberta)
#rep.report()

In [24]:
#gpuinfo()

In [25]:
#del model
#del optimizer 
#del train_loader
#del val_loader
#del scheduler 
#del list_val_rmse
#del train_indices
#del val_indices
#del tokenizer
#torch.cuda.empty_cache()
#gpuinfo()

# Inference

In [26]:
%cd
!mkdir .kaggle
!mkdir /content/model
!cp /content/drive/MyDrive/Colab_Files/kaggle-api/kaggle.json .kaggle/

!cp -r /content/model_1.pth /content/model/model_1.pth
!cp -r /content/model_2.pth /content/model/model_2.pth
!cp -r /content/model_3.pth /content/model/model_3.pth
!cp -r /content/model_4.pth /content/model/model_4.pth
!cp -r /content/model_5.pth /content/model/model_5.pth

/root


In [27]:


def dataset_upload():
    import json
    from kaggle.api.kaggle_api_extended import KaggleApi

    id = f'{USERID}/{EX_NO}'

    dataset_metadata = {}
    dataset_metadata['id'] = id
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = f'{EX_NO}'

    with open(UPLOAD_DIR / 'dataset-metadata.json', 'w') as f:
        json.dump(dataset_metadata, f, indent=4)

    api = KaggleApi()
    api.authenticate()

    # データセットがない場合
    if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
        api.dataset_create_new(folder=UPLOAD_DIR,
                               convert_to_csv=False,
                               dir_mode='skip')
    # データセットがある場合
    else:
        api.dataset_create_version(folder=UPLOAD_DIR,
                                   version_notes='update',
                                   convert_to_csv=False,
                                   delete_old_versions=True,
                                   dir_mode='skip')
dataset_upload()



Starting upload for file model_3.pth


100%|██████████| 477M/477M [00:49<00:00, 10.0MB/s]


Upload successful: model_3.pth (477MB)
Starting upload for file model_2.pth


100%|██████████| 477M/477M [00:32<00:00, 15.3MB/s]


Upload successful: model_2.pth (477MB)
Starting upload for file model_5.pth


...

Upload successful: model_5.pth (477MB)
Starting upload for file model_1.pth


100%|██████████| 477M/477M [00:37<00:00, 13.3MB/s]


Upload successful: model_1.pth (477MB)
Starting upload for file model_4.pth


100%|██████████| 477M/477M [00:34<00:00, 14.4MB/s]


Upload successful: model_4.pth (477MB)
